In [1]:
from helper import *
from postProcessHelper import *

varsDictP1 = setUpPostProcessVars(versionControl='V3', promptVersion='P1', simpleCourseName='MOVESCI')
varsDictP2 = setUpPostProcessVars(versionControl='V3', promptVersion='P2', simpleCourseName='MOVESCI')

resultsDFCols = ['submitter_id', 'grader_id', 'rubric_id', 'assignment_id', 'score', 'points_possible']

config = Config()
config.setFromEnv()

versionControl = 'V3'
promptVersion='P2'
config.simpleCourseName = 'MOVESCI'
fullName = f'{config.simpleCourseName}-{versionControl}-{promptVersion}'

config.setSaveDetails(fullName)

excelFolder = os.path.join(config.baseOutputFolder, \
                           config.outputFolders['EXCEL_OUTPUT'], \
                           config.fullName)

compareResultsDF = varsDictP1['resultsDF'].merge(varsDictP2['resultsDF'], \
                                                 on=resultsDFCols, how='inner', \
                                                 suffixes=('_P1', '_P2'))

2023-07-11T10:57:00-0400 INFO [helper.py:417] - All configuration parameters set up successfully.
2023-07-11T10:57:02-0400 INFO [helper.py:417] - All configuration parameters set up successfully.
2023-07-11T10:57:04-0400 INFO [helper.py:417] - All configuration parameters set up successfully.


In [3]:
fullInfoDF, rubricOrderDict = buildComparerFullInfoDF(config, compareResultsDF, saveName, excelFolder)

In [13]:
def getHighErrorCriteria(config, fullInfoDF, saveName, excelFolder, scoreThreshold = 10):
    pd.set_option('display.precision', 2)

    descDataDF = config.critDescDF[['custom_description', 'assignment_id', 'id']]
    criteriaIssuesDF = pd.DataFrame()
    P1Count,P2Count = 0,0
    binaryTracker = []
    for AID in fullInfoDF['assignment_id'].unique():
        for CID in fullInfoDF[(fullInfoDF['assignment_id']==AID)]['criterion_id'].unique(): 
            subsetDF =  fullInfoDF[(fullInfoDF['assignment_id']==AID) \
                                   & (fullInfoDF['criterion_id']==CID)]
            if subsetDF['Mean Difference % P1'].mean() > scoreThreshold:
                issueDF = subsetDF[['assignment_id', 'criterion_id', \
                                    'assignment_title', 'description_rubric', \
                                    'points_rubric', 'All Graders Mean', \
                                    'Global peerGPT Mean P1']].drop_duplicates()
                issueDF['Mean Difference % P1'] = subsetDF['Mean Difference % P1'].mean()
                P1Count += 1

            if subsetDF['Mean Difference % P2'].mean() > scoreThreshold:
                issueDF = subsetDF[['assignment_id', 'criterion_id', \
                                    'assignment_title', 'description_rubric', \
                                    'points_rubric', 'All Graders Mean', \
                                    'Global peerGPT Mean P2']].drop_duplicates()
                issueDF['Mean Difference % P2'] = subsetDF['Mean Difference % P2'].mean()
                P2Count += 1
            
            binaryTracker.append(str(int(subsetDF['Mean Difference % P1'].mean() > scoreThreshold))\
                               + str(int(subsetDF['Mean Difference % P2'].mean() > scoreThreshold)))
            
            criteriaIssuesDF = pd.concat([criteriaIssuesDF, issueDF])

    criteriaIssuesDF = criteriaIssuesDF.merge(descDataDF, left_on=['assignment_id', 'criterion_id'], \
                                            right_on=['assignment_id', 'id'])
    del criteriaIssuesDF['id']
    criteriaIssuesDF = criteriaIssuesDF.rename(columns={'assignment_title':'Title', \
                                                        'custom_description':'Supplemental Description', \
                                                        'description_rubric':'Rubric', \
                                                        'points_rubric':'Max Score'}).reset_index(drop=True)
    criteriaIssuesDF.to_excel(os.path.join(excelFolder, saveName+' - High Error Criteria.xlsx'))

    return criteriaIssuesDF, {'With no guide':P1Count, 'With grading guide':P2Count}, binaryTracker

criteriaIssuesDF, hitCounts, binaryTracker = getHighErrorCriteria(config, fullInfoDF, saveName, excelFolder, scoreThreshold = 10)

binaryTracker

['11',
 '11',
 '10',
 '11',
 '00',
 '00',
 '00',
 '00',
 '00',
 '00',
 '00',
 '11',
 '00',
 '00',
 '00',
 '11',
 '00',
 '00',
 '00',
 '00',
 '01',
 '00',
 '00',
 '10',
 '11',
 '10',
 '11',
 '11',
 '11',
 '00',
 '00',
 '11',
 '00',
 '00',
 '00',
 '00']